# embedding

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
from pathlib import Path

# Get list of stored filed in cloud bucket
root = Path(os.path.expanduser("~"))
! date

Sat Mar 15 17:28:29 EDT 2025


In [8]:
# path to data
data_path = f"{root}/github/pytorch-plantclef/data/subset_top5_train"

# read train data
train_df = pd.read_parquet(data_path)
# schema
print(train_df.shape)
train_df.head(3)

(3760, 22)


,species_id,image_name,path,data,organ,obs_id,license,partner,author,altitude,...,gbif_species_id,species,genus,family,dataset,publisher,references,url,learn_tag,image_backup_url
0,1360257,5a4deafc321f2b653f03cc238160a799b0bbb6b7.jpg,/PlantCLEF2024/train/1360257/5a4deafc321f2b653...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,bark,1015292920,cc-by-sa,None,Gabor Graehn,NaN,...,3039454.0,Frangula alnus Mill.,Frangula,Rhamnaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/5a4deafc321f2b...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
1,1360257,a6d9a13ec26cbb04d61861ad5cb4ede8eb4d1ea3.jpg,/PlantCLEF2024/train/1360257/a6d9a13ec26cbb04d...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,bark,1005087257,cc-by-sa,None,Jacco Sanders,51.0,...,3039454.0,Frangula alnus Mill.,Frangula,Rhamnaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/a6d9a13ec26cbb...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
2,1360257,0fec55fc5105e512df08490ecbb55df3393d85c8.jpg,/PlantCLEF2024/train/1360257/0fec55fc5105e512d...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,leaf,4111721487,http://creativecommons.org/licenses/by-nc/4.0/,None,imrsmf,NaN,...,3039454.0,Frangula alnus Mill.,Frangula,Rhamnaceae,gbif,iNaturalist,https://www.inaturalist.org/photos/268971946,https://inaturalist-open-data.s3.amazonaws.com...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
